Import thư viện

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Load và chia dữ liệu cho quá trình huấn luyện

In [ ]:
# Kích thước ảnh và batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

# Đường dẫn đến thư mục chứa dữ liệu
data_dir = '/content/drive/MyDrive/ISIC_2018/'
images_dir = os.path.join(data_dir, 'ISIC2018_Task3_Training_Input/ISIC2018_Task3_Training_Input')
labels_path = os.path.join(data_dir, 'ISIC2018_Task3_Training_GroundTruth.csv')


In [ ]:
total_images = df.shape[0]
print(f"Tổng số ảnh: {total_images}")


Tổng số ảnh: 10015


In [ ]:
# Đọc file labels.csv
df = pd.read_csv(labels_path)

# Xác định cột nhãn (label) từ các cột chẩn đoán
diagnosis_columns = df.columns[1:]  # Bỏ cột đầu tiên là 'image'

# Tạo cột 'label' chứa tên loại bệnh tương ứng
df['label'] = df[diagnosis_columns].idxmax(axis=1)

# Chia dữ liệu thành train (75%) và temp (25%)
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, test_size=0.25, stratify=df['label'], random_state=42)

# Chia temp thành validation (15%) và test (10%)
val_df, test_df = train_test_split(temp_df, test_size=0.4, stratify=temp_df['label'], random_state=42)


In [ ]:
train_df['image'] = train_df['image'].apply(lambda x: x if x.endswith('.jpg') else x + '.jpg')
val_df['image'] = val_df['image'].apply(lambda x: x if x.endswith('.jpg') else x + '.jpg')
test_df['image'] = test_df['image'].apply(lambda x: x if x.endswith('.jpg') else x + '.jpg')


In [ ]:
print(f"Tổng số ảnh trong tập huấn luyện (train): {len(train_df)}")
print(f"Tổng số ảnh trong tập xác thực (validation): {len(val_df)}")
print(f"Tổng số ảnh trong tập kiểm tra (test): {len(test_df)}")


Tổng số ảnh trong tập huấn luyện (train): 7511
Tổng số ảnh trong tập xác thực (validation): 1502
Tổng số ảnh trong tập kiểm tra (test): 1002


In [ ]:
print(df.columns)


Index(['image', 'MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC', 'label'], dtype='object')


Xử lý mất cân bằng dữ liệu bằng trọng số weights

In [ ]:
# Tính toán class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)

# Chuyển class_weights thành dictionary
class_weights_dict = dict(zip(np.unique(train_df['label']), class_weights))


Tăng cường dữ liệu

In [ ]:
# Tăng cường dữ liệu cho tập huấn luyện
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,          #Xoay ảnh ngẫu nhiên 30 độ
    width_shift_range=0.1,      #Dịch ngang ảnh max 10% chiều rộng
    height_shift_range=0.1,     #Dịch dọc ảnh max 10% chiều cao
    shear_range=0.2,            #Xoay nghiêng 20%
    zoom_range=0.2,             #Phóng thu 20%
    horizontal_flip=True,
    fill_mode='nearest'
)

# Chỉ rescale cho tập validation và test
val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Tạo generator cho từng tập
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=images_dir,
    x_col='image',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_dataframe(
    val_df,
    directory=images_dir,
    x_col='image',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_dataframe(
    test_df,
    directory=images_dir,
    x_col='image',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 7511 validated image filenames belonging to 7 classes.
Found 1502 validated image filenames belonging to 7 classes.
Found 1002 validated image filenames belonging to 7 classes.


Xây dựng mô hình phân loại bằng cách fine-tune VGG16

In [ ]:
# Số lượng lớp (số loại bệnh)
num_classes = df['label'].nunique()

# Tải mô hình VGG16 với trọng số từ ImageNet, không bao gồm các lớp fully connected ở trên cùng
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Đóng băng các lớp convolutional
for layer in base_model.layers:
    layer.trainable = False

# Thêm các lớp fully connected tùy chỉnh
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Tạo mô hình hoàn chỉnh
model = Model(inputs=base_model.input, outputs=predictions)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Huấn luyện mô hình và lưu checkpoint vào drive

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

# Biên dịch mô hình
optimizer = Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Mount Google Drive (nếu chưa mount)
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục nếu chưa tồn tại
import os
checkpoint_dir = '/content/drive/MyDrive/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Định nghĩa các callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    os.path.join(checkpoint_dir, 'best_model.h5'),
    save_best_only=True
)

# Huấn luyện mô hình
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=[early_stop, model_checkpoint]
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.2310 - loss: 2.8082

470/470 ━━━━━━━━━━━━━━━━━━━━ 2696s 6s/step - accuracy: 0.2311 - loss: 2.8078 - val_accuracy: 0.5087 - val_loss: 1.7214
Epoch 2/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step - accuracy: 0.3763 - loss: 2.3475

470/470 ━━━━━━━━━━━━━━━━━━━━ 187s 398ms/step - accuracy: 0.3764 - loss: 2.3474 - val_accuracy: 0.5366 - val_loss: 1.6623
Epoch 3/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step - accuracy: 0.4279 - loss: 2.0561

470/470 ━━━━━━━━━━━━━━━━━━━━ 182s 385ms/step - accuracy: 0.4279 - loss: 2.0561 - val_accuracy: 0.5965 - val_loss: 1.4550
Epoch 4/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step - accuracy: 0.4573 - loss: 1.9937

470/470 ━━━━━━━━━━━━━━━━━━━━ 177s 377ms/step - accuracy: 0.4574 - loss: 1.9936 - val_accuracy: 0.6391 - val_loss: 1.3262
Epoch 5/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step - accuracy: 0.4945 - loss: 1.8494

...

470/470 ━━━━━━━━━━━━━━━━━━━━ 180s 382ms/step - accuracy: 0.7321 - loss: 0.9000 - val_accuracy: 0.7756 - val_loss: 0.7768
Epoch 30/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step - accuracy: 0.7399 - loss: 0.8728

470/470 ━━━━━━━━━━━━━━━━━━━━ 194s 367ms/step - accuracy: 0.7400 - loss: 0.8727 - val_accuracy: 0.7736 - val_loss: 0.7533
Epoch 31/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 201s 365ms/step - accuracy: 0.7459 - loss: 0.8141 - val_accuracy: 0.7750 - val_loss: 0.7700
Epoch 32/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step - accuracy: 0.7567 - loss: 0.7742

470/470 ━━━━━━━━━━━━━━━━━━━━ 173s 368ms/step - accuracy: 0.7567 - loss: 0.7742 - val_accuracy: 0.7816 - val_loss: 0.7290
Epoch 33/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 201s 365ms/step - accuracy: 0.7613 - loss: 0.7607 - val_accuracy: 0.7790 - val_loss: 0.7414
Epoch 34/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 171s 365ms/step - accuracy: 0.7549 - loss: 0.7601 - val_accuracy: 0.7810 - val_loss: 0.7329
Epoch 35/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 172s 366ms/step - accuracy: 0.7662 - loss: 0.7603 - val_accuracy: 0.7776 - val_loss: 0.7293
Epoch 36/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 174s 370ms/step - accuracy: 0.7698 - loss: 0.7345 - val_accuracy: 0.7909 - val_loss: 0.7310
Epoch 37/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - accuracy: 0.7787 - loss: 0.7448

470/470 ━━━━━━━━━━━━━━━━━━━━ 178s 378ms/step - accuracy: 0.7787 - loss: 0.7448 - val_accuracy: 0.7816 - val_loss: 0.7210
Epoch 38/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step - accuracy: 0.7745 - loss: 0.7207

470/470 ━━━━━━━━━━━━━━━━━━━━ 177s 376ms/step - accuracy: 0.7745 - loss: 0.7207 - val_accuracy: 0.7843 - val_loss: 0.6980
Epoch 39/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 180s 383ms/step - accuracy: 0.7787 - loss: 0.6931 - val_accuracy: 0.7870 - val_loss: 0.7262
Epoch 40/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 173s 368ms/step - accuracy: 0.7708 - loss: 0.7008 - val_accuracy: 0.7916 - val_loss: 0.7029
Epoch 41/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 174s 369ms/step - accuracy: 0.7721 - loss: 0.7072 - val_accuracy: 0.7896 - val_loss: 0.7155
Epoch 42/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 173s 368ms/step - accuracy: 0.7858 - loss: 0.6698 - val_accuracy: 0.7916 - val_loss: 0.7030
Epoch 43/50
470/470 ━━━━━━━━━━━━━━━━━━━━ 173s 368ms/step - accuracy: 0.7911 - loss: 0.6495 - val_accuracy: 0.7929 - val_loss: 0.7036


In [ ]:
# Đánh giá mô hình trên tập test
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Loss trên tập test: {test_loss}')
print(f'Độ chính xác trên tập test: {test_accuracy}')


63/63 ━━━━━━━━━━━━━━━━━━━━ 284s 5s/step - accuracy: 0.7934 - loss: 0.7469
Loss trên tập test: 0.6680651903152466
Độ chính xác trên tập test: 0.7984032034873962
